<img src="uva_seal.png">  

## MapReduce Framework

### University of Virginia
### DS 5110: Big Data Systems
### Last Updated: January 14, 2026

---  

### SOURCE

Hadoop: The Definitive Guide, Tom White

### OBJECTIVES
-  Introduction to the MapReduce framework

### CONCEPTS

- Centralized vs Decentralized vs Distributed Data
- MapReduce
- Jobs
- Tasks
- Map operation
- Reduce operation
- Shuffle operation

---

### I. Centralized vs Decentralized vs Distributed Data

We start by explaining *centralized*, *decentralized*, and *distributed data*.

In this course, we focus on distributed data.

Imagine we have a large amount of data and we are deciding how it should be stored and managed.

*Centralized data* has a single control point (it may live on a single large machine).  
Managing this data may be relatively easy, but there is a single failure point.  
It also won't scale.

*Decentralized data* is spread across machines (nodes).   
A decentralized system has **no single controlling authority**. Each node is autonomous and may have its own goals, policies, or owners.  
This can help scale, but it will be more complex to manage.  
An example would be a set of iPhones collecting data, where no single server collects and manages the data.  

*Distributed data* is spread across nodes.  
There is a **single controlling authority**, which helps coordinate activities.  
Nodes cooperate to act like one large computer.  
Spark is a distributed system.

<img src="data_pattern.png" width=600>

---


## II. Introducing MapReduce

`MapReduce` is a programming framework for processing large data sets with a parallel, **distributed algorithm** on a cluster.  

First the data set is “mapped” into a collection of `<key, value>` pairs, and then “reduced” over all pairs with the same key.  
We need to discuss what this means.  

The operations are surprisingly broad, enabling them to handle a wide range of use cases.

The popular minimal example is to do a word count on some text.

Initially published in 2004 by employees at Google

A `MapReduce` job is a unit of work the client wants to be performed  

A job consists of:  

- Input data
- `MapReduce` program
- configs

The job will be divided into *tasks*  
Two types of tasks: *map tasks* and *reduce tasks*

---

**Aside on `Hadoop`**  

`MapReduce` forms the computation paradigm for Hadoop  

`Hadoop` was created by Doug Cutting and Mike Cafarella.  Version 1 was called Nutch.  

Yahoo! was instrumental in providing a dedicated team and resources to turn Hadoop into a system that ran at web scale.

Hadoop is developed and maintained by the Apache Software Foundation.


**Aside on Hadoop and Spark**  

Spark does not follow the `MapReduce` framework in the same way that `Hadoop` does, where `Mapper` and `Reducer` functions are detailed explicitly.  

This is actually a big advantage of Spark.

Spark and Hadoop can be better together.  For example, some teams will use Hadoop data storage (HDFS) with Spark.  
That is, a Spark job can read data from HDFS, and write results to HDFS.

Reported runtimes:
Spark can be as much as 10x faster than `MapReduce` for batch processing, and up to 100x faster for in-memory analytics.

The backbone of this course will cover Spark (in particular, PySpark) for various tasks.

Next we illustrate the `MapReduce` framework with an example.

---

**MapReduce Word Count Process Diagram**

<img src="map_reduce_example2.png">

**Some Things to Notice:**

**Splitting Step**  
A given number of records is assigned to each worker

NOTE:  
More splits means less processing time per split  
However, the overhead of managing splits and `map` task creation starts to dominate total job execution time  

**Mapping Step**  
Each token (word) is given a value of 1 representing the count.  
The token, count forms a `<key, value> pair.`  
If the same pair appears $n$ times on a worker, there would be $n$ entries of `<map, 1>`.  In other words, no aggregation is done at this step (it happens at the `reduce` step).  

**Shuffling Step**  
The `<key, value>` pairs are moved across machines at this point, so that pairs with the same key are gathered on the same machine.  This is a costly step in the process.

**Reducing Step**  
A `reducer` applies a given operation over the values by key.  For this word count example, the counts would be summed for each key and stored with the key.

**Map and Reduce in General**  

**Map**  
Higher-order function that applies a function to each element of a list (e.g., $x => x**2$)

**Reduce**  
Higher-order function that applies a combining operation (e.g,. cumulative sum of values)

**Combiner Functions**  
It can be expensive to shuffle data within the cluster, and the limiting factor is the bandwidth on the cluster.

`Combiner Functions` can be run on the map output, before the shuffle occurs. This acts locally on each node, and reduces some of the data transfer.  Note this won’t always work (e.g., can’t be used for computing the average, but CAN be used for computing the maximum).

---


### III. MapReduce Code Illustration

Next, we look at how PySpark implements the word count program.  

**Word Count Example**  
Data source: Spark README file

In [ ]:
# import libraries
from pyspark.sql import SparkSession
import os

# context manager: minimal settings
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

In [ ]:
# list files in the directory
os.listdir()

In [ ]:
os.getcwd()

In [ ]:
# Read in data
lines = sc.textFile("README.txt")

In [ ]:
type(lines)

In [ ]:
lines.count()

In [ ]:
# show some records
lines.take(5)

**Show top 10 most frequent words** (we will examine in detail later)

In [ ]:
words = lines.flatMap(lambda x: x.split())
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)
wordcounts.take(10)

**Question: How are `map()` and `flatMap()` different?**

In [ ]:
# Distribute data to the workers
data = sc.parallelize([3,4,5])

type(data)

In [ ]:
sc.parallelize([3,4,5]).map(lambda x: [x,  x*x]).collect() 

In [ ]:
sc.parallelize([3,4,5]).flatMap(lambda x: [x, x*x]).collect() 

map: a list of lists  
flatMap: a flattened list

---

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Copy the word count code into the cell below.  Modify it to print the top 20 wordcount pairs.

2) Copy the word count code into the cell below.  Modify it to filter out rows containing the word *Spark* and then execute the word count, returning the top 10 wordcount pairs.

**SOLUTIONS**

In [ ]:
# 1)

wordcounts = lines.flatMap(lambda x: x.split()) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(False)

wordcounts.take(20)

In [ ]:
# 2)

wordcounts = lines.filter(lambda x: "Spark" not in x) \
            .flatMap(lambda x: x.split()) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(False)

wordcounts.take(10)